In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle



    Only loading the 'en' tokenizer.



In [2]:
PATH='data/predictHappiness/'
!ls {PATH}

all
models
PredictHappinessModel_17_11_22_01.csv
PredictHappinessModel_17_11_23_01.csv
PredictHappinessModel_17_11_24_01.csv
PredictHappinessModel_17_11_24_02.csv
PredictHappinessModel_17_11_24_03.csv
PredictHappinessModel_17_11_25_01.csv
PredictHappinessModel_17_11_26_01_13.csv
PredictHappinessModel_17_11_26_01_15.csv
PredictHappinessModel_17_11_26_03.csv
PredictHappinessModel_17_11_26_04.csv
PredictHappinessModel_17_11_26_05.csv
PredictHappinessModel_17_11_27_01.csv
PredictHappinessModel_17_11_27_02.csv
PredictHappinessModel_17_11_27_03.csv
PredictHappinessModel_17_11_27_04.csv
PredictHappinessModel_17_11_27_05_ensemble.csv
sample_submission.csv
Stage1Model
Stage2Model
test.csv
tmp
train.csv
trn
val


In [3]:
trn = pd.read_csv(PATH+"train.csv")
tst = pd.read_csv(PATH+"test.csv")

In [4]:
for i in trn.Is_Response.unique():
    trn_dir_path = f'{PATH}trn/{i.replace(" ","_")}'
    val_dir_path = f'{PATH}val/{i.replace(" ","_")}'
    !rm -r {trn_dir_path}
    !rm -r {val_dir_path}
    os.makedirs(trn_dir_path, exist_ok=True)
    os.makedirs(val_dir_path, exist_ok=True)
!rm -r {PATH}all/trn
!rm -r {PATH}all/val
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}all/tst', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)
os.makedirs(f'{PATH}/models/Stage1Model', exist_ok=True)
os.makedirs(f'{PATH}/models/Stage2Model', exist_ok=True)

This saves the csv in individual files in folders that split them into happy and not_happy


In [5]:
for i in range(trn.values[:,1].shape[0]):
    dset = 'trn' if random.random()>0.05 else 'val'
    f = open(f"{PATH}all/{dset}/"+trn.values[i,0]+".txt", 'w')
    f.write(trn.values[i,1])
    f.close()
    f = open(f"{PATH}{dset}/"+trn.Is_Response[i].replace(" ","_")+"/"+trn.values[i,0]+".txt", 'w')
    f.write(trn.values[i,1])
    f.close()

In [6]:
for i in range(tst.values[:,1].shape[0]):
    f = open(PATH+"all/tst/"+tst.values[i,0]+".txt", 'w')
    f.write(tst.values[i,1])
    f.close()

In [14]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [9]:
bs=64; bptt=70

In [15]:
FILES = dict(train="all/trn/", validation="all/val/", test="all/tst/")

In [16]:
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [11]:
#pickle.dump(TEXT, open(f'{PATH}models/TEXT3.pkl','wb'))

In [12]:
TEXT.vocab.itos

['<unk>',
 '<pad>',
 '"',
 'the',
 '.',
 ',',
 'and',
 'a',
 'to',
 'was',
 'i',
 'in',
 'we',
 'of',
 'is',
 'for',
 'hotel',
 'it',
 '-',
 'room',
 'at',
 'were',
 'that',
 'but',
 'on',
 'this',
 'with',
 'you',
 'very',
 'not',
 '<eos>',
 '!',
 'had',
 'there',
 'my',
 'our',
 'are',
 'have',
 ')',
 'they',
 '(',
 'from',
 'as',
 'stay',
 "n't",
 'great',
 'staff',
 'so',
 'would',
 'be',
 'all',
 'rooms',
 'if',
 "'s",
 'did',
 'one',
 '--',
 'good',
 'location',
 'nice',
 'out',
 'no',
 'stayed',
 'when',
 'which',
 'clean',
 'an',
 'us',
 'night',
 'just',
 'service',
 'or',
 'only',
 'breakfast',
 'up',
 'get',
 'here',
 'do',
 'about',
 'could',
 'me',
 'time',
 'also',
 'by',
 '$',
 'desk',
 'like',
 'bed',
 'again',
 'friendly',
 'well',
 'area',
 'place',
 'other',
 'day',
 'comfortable',
 'front',
 'small',
 'some',
 'really',
 'can',
 'will',
 'back',
 'more',
 'free',
 'floor',
 'bathroom',
 'your',
 'even',
 'after',
 'helpful',
 'check',
 'what',
 'two',
 '...',
 'than

In [17]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [18]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [43]:
multiplier=1
dropouti=0.05*multiplier
dropout=0.05*multiplier
wdrop=0.1*multiplier
dropoute=0.02*multiplier
dropouth=0.05*multiplier
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=dropouti, dropout=dropout, wdrop=wdrop, dropoute=dropoute, dropouth=dropouth)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [44]:
learner.fit(3e-3, 7, wds=1e-6, cycle_len=1, cycle_mult=2, cycle_save_name="PtH_Stage1_cyclemult")

HBox(children=(IntProgress(value=0, description='Epoch', max=127), HTML(value='')))

[ 0.       4.65618  4.6425 ]                                  
[ 1.       3.83107  3.76976]                                  
[ 2.       3.68609  3.64447]                                  
[ 3.       3.59806  3.52884]                                  
[ 4.       3.47371  3.44122]                                  
[ 5.       3.40985  3.38482]                                  
[ 6.       3.36285  3.37347]                                  
[ 7.       3.45098  3.41205]                                  
[ 8.       3.40512  3.37419]                                  
[ 9.       3.36023  3.34766]                                  
[ 10.        3.33518   3.31689]                               
[ 11.        3.2799    3.28944]                               
[ 12.        3.24986   3.28134]                               
[ 13.        3.22152   3.27062]                               
[ 14.        3.21745   3.26814]                               
[ 15.        3.35704   3.33033]                        

In [55]:
learner.load_cycle("PtH_Stage1_cyclemult", 6)

In [56]:
learner.save_encoder('PtH_Stage1_cyclemult_newtest')

In [63]:
learner.fit(3e-3, 7, wds=1e-6, cycle_len=1, cycle_mult=2, cycle_save_name="PtH_Stage1_cyclemul2")

HBox(children=(IntProgress(value=0, description='Epoch', max=127), HTML(value='')))

[ 0.       3.0863   3.18537]                                  
[ 1.       3.15896  3.203  ]                                  
[ 2.       3.0592   3.18375]                                  
[ 3.       3.20134  3.22471]                                  
[ 4.       3.12271  3.19833]                                  
[ 5.       3.05572  3.1796 ]                                  
[ 6.       3.01982  3.17836]                                  
[ 7.       3.19351  3.22282]                                  
[ 8.       3.16917  3.22355]                                  
[ 9.       3.16295  3.2084 ]                                  
[ 10.        3.11239   3.19955]                               
[ 11.        3.06802   3.18485]                               
[ 12.        3.02773   3.18085]                               
[ 13.        3.01139   3.17454]                               
[ 14.        3.00529   3.17281]                               
[ 15.        3.19182   3.22998]                        

In [104]:
learner.load_cycle("PtH_Stage1_cyclemul2", 6)

In [66]:
learner.save_encoder('PtH_Stage1_cyclemult_newtest_secondRun')

In [105]:
learner.fit(3e-3, 8, wds=1e-6, cycle_len=1, cycle_mult=2, cycle_save_name="PtH_Stage1_cyclemult3")

HBox(children=(IntProgress(value=0, description='Epoch', max=255), HTML(value='')))

[ 0.       3.04982  3.17779]                                  
[ 1.       3.12451  3.18831]                                  
[ 2.       3.04643  3.16842]                                  
[ 3.       3.16214  3.20871]                                  
[ 4.       3.09606  3.18544]                                  
[ 5.       3.03946  3.16201]                                  
[ 6.       2.99206  3.16952]                                  
[ 7.       3.17405  3.21247]                                  
[ 8.       3.14598  3.20541]                                  
[ 9.       3.1541   3.19175]                                  
[ 10.        3.06546   3.18387]                               
[ 11.        3.03504   3.16886]                               
[ 12.        3.01558   3.16507]                               
[ 13.        2.9809    3.16332]                               
[ 14.        2.98458   3.16503]                               
[ 15.        3.16535   3.20943]                        

[ 130.         3.12115    3.20805]                            
[ 131.         3.13842    3.2031 ]                            
[ 132.         3.13271    3.2052 ]                            
[ 133.         3.12759    3.20495]                            
[ 134.         3.13595    3.2012 ]                            
[ 135.         3.12336    3.20129]                            
[ 136.         3.13348    3.2021 ]                            
[ 137.         3.11861    3.20034]                            
[ 138.         3.1395     3.21332]                            
[ 139.         3.11951    3.19423]                            
[ 140.         3.11631    3.20261]                            
[ 141.         3.12162    3.20102]                            
[ 142.         3.11319    3.20614]                            
[ 143.         3.12963    3.19646]                            
[ 144.         3.11703    3.19683]                            
[ 145.         3.11195    3.19966]                     

In [106]:
learner.load_cycle("PtH_Stage1_cyclemult3", 7)

In [107]:
learner.save_encoder('PtH_Stage1_cyclemult_newtest_thirdRun')

In [4]:
TEXT = pickle.load(open(f'{PATH}models/TEXT3.pkl','rb'))

In [16]:
TEXT.vocab

In [108]:
m=learner.model
ss=""".  Breakfast was the worst"""
s = [spacy_tok(ss)]
t = TEXT.numericalize(s)
' '.join(s[0])

'.   Breakfast was the worst'

In [109]:
m[0].bs = 1
m.eval()
m.reset()
res,*_ = m(t)

m[0].bs=bs

In [110]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['i',
 'hotel',
 'experience',
 'part',
 'thing',
 'breakfast',
 'service',
 'customer',
 '<unk>',
 'we']

In [111]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

.  Breakfast was the worst 

i have ever had . the staff was very rude and the hotel was not clean . i would not recommend this hotel to anyone . <eos> i stayed at the hotel for - nights in early september . the hotel is in a great location , close to the ...


In [19]:
class PredictHappinessDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [("Description", text_field), ("Is_Response", label_field)]
        examples = []
        #for label in ['happy', 'not_happy']:
        #    for fname in iglob(os.path.join(path, label, '*.txt')):
        #        with open(fname, 'r') as f: text = f.readline()
        for i in range(trn.values[:,1].shape[0]):
            text = trn.Description[i]
            label = trn.Is_Response[i]
            examples.append(data.Example.fromlist([text, label], fields))#[fields[1], fields[-1]]))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.Description)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [20]:
PH_LABEL = data.Field(sequential=False)

In [21]:
splits = PredictHappinessDataset.splits(TEXT, PH_LABEL, PATH, train='trn', test='val')

In [22]:
md2 = TextData.from_splits(PATH, splits, bs=bs, text_name="Description", label_name="Is_Response")

In [41]:
multiplier=1
dropout=0.1*multiplier
dropouti=0.4*multiplier
wdrop=0.5*multiplier
dropoute=0.05*multiplier
dropouth=0.3*multiplier

m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl,dropout=dropout, dropouti=dropouti, wdrop=wdrop, dropoute=dropoute, dropouth=dropouth)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('PtH_Stage1_cyclemult_newtest_secondRun')

m3.clip=25.
lrs=np.array([1e-4,1e-3,1e-2])

In [42]:
#this is from the second run through doubling dictionary traintime
m3.freeze_to(-1)
m3.fit(lrs, 2, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       0.45891  0.38741  0.8448 ]                        
[ 1.       0.44171  0.29139  0.88068]                        



HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

[ 0.       0.30242  0.24222  0.90412]                        
[ 1.       0.28909  0.2235   0.91198]                        
[ 2.       0.2706   0.22141  0.91319]                        
[ 3.       0.27693  0.21516  0.91761]                        
[ 4.       0.25197  0.2092   0.91833]                        
[ 5.       0.23405  0.20226  0.92193]                        
[ 6.       0.23794  0.20145  0.92257]                        
[ 7.       0.24496  0.20038  0.92211]                        
[ 8.       0.2374   0.21363  0.91823]                        
[ 9.       0.23678  0.18677  0.92694]                        
[ 10.        0.23004   0.18318   0.9282 ]                    
[ 11.        0.21947   0.17611   0.93118]                    
[ 12.        0.213     0.17358   0.93272]                    
[ 13.        0.20926   0.17303   0.93305]                    
[ 14.        0.21238   0.17241   0.93318]                    



In [101]:
m3.save_encoder("m3_weights")

In [43]:
m3.save("m3_model2")

In [24]:
m3.load("m3_model")

In [25]:
multiplier=1
dropout=0.1*multiplier
dropouti=0.4*multiplier
wdrop=0.5*multiplier
dropoute=0.05*multiplier
dropouth=0.3*multiplier

m4 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl,dropout=dropout, dropouti=dropouti, wdrop=wdrop, dropoute=dropoute, dropouth=dropouth)
m4.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m4.load_encoder('PtH_Stage1_cyclemult_newtest')

m4.clip=25.

In [97]:
m4.freeze_to(-1)
m4.fit(lrs, 4, metrics=[accuracy])
m4.unfreeze()
m4.fit(lrs, 4, metrics=[accuracy], cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       0.40545  0.32259  0.87009]                        
[ 1.       0.40197  0.36688  0.85231]                        
[ 2.       0.43458  0.27963  0.8843 ]                        
[ 3.       0.4265   0.28263  0.88567]                        



HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

[ 0.       0.31343  0.24236  0.90365]                        
[ 1.       0.28469  0.22289  0.91142]                        
[ 2.       0.26625  0.2234   0.91293]                        
[ 3.       0.26935  0.22339  0.91224]                        
[ 4.       0.25794  0.2094   0.91856]                        
[ 5.       0.23912  0.20621  0.92018]                        
[ 6.       0.23435  0.20324  0.92126]                        
[ 7.       0.25121  0.19627  0.92301]                        
[ 8.       0.24058  0.19371  0.92254]                        
[ 9.       0.23593  0.1848   0.92768]                        
[ 10.        0.22597   0.17993   0.92948]                    
[ 11.        0.22256   0.17637   0.93118]                    
[ 12.        0.2095    0.17469   0.93177]                    
[ 13.        0.21815   0.17338   0.9321 ]                    
[ 14.        0.20402   0.17356   0.93187]                    



In [102]:
m4.save_encoder("m4_weights")

In [26]:
m4.save("m4_model")

In [47]:
multiplier=1
dropout=0.1*multiplier
dropouti=0.4*multiplier
wdrop=0.5*multiplier
dropoute=0.05*multiplier
dropouth=0.3*multiplier

m5 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl,dropout=dropout, dropouti=dropouti, wdrop=wdrop, dropoute=dropoute, dropouth=dropouth)
m5.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m5.load_encoder('PtH_Stage1_cyclemult_newtest_thirdRun')

m5.clip=25.

In [49]:
m5.freeze_to(-1)
m5.fit(lrs, 1, metrics=[accuracy])
m5.unfreeze()
m5.fit(lrs, 3, metrics=[accuracy], cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       0.23662  0.20586  0.92059]                        



HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.24183  0.19892  0.92267]                        
[ 1.       0.23107  0.19165  0.92509]                        
[ 2.       0.22272  0.1898   0.92599]                        
[ 3.       0.22012  0.18766  0.92668]                        
[ 4.       0.22558  0.17965  0.92887]                        
[ 5.       0.21583  0.17787  0.93128]                        
[ 6.       0.23316  0.17673  0.93126]                        



In [50]:
m5.save("m5_model2")

In [51]:
submission = pd.DataFrame(columns=['User_ID','Is_Response'])

In [82]:
m3m = m3.model
m3m[0].bs = 1
ss = tst["Description"][0]
s = [spacy_tok(ss)]
t = TEXT.numericalize(s)
   
m3m.eval()
m3m.reset()
resm3m,*_ = m3m(t)

In [84]:
ss

"Looking for a motel in close proximity to TV taping of a Dr. Phil show, we chose the Dunes on Sunset Blvd in West Hollywood. Although the property displayed the AAA emblem, it certainly left a lot to be desired. There were chips & scrapes on the bottom of the door frame in the bathroom and the lotion containers were half full--apparently not replaced by housekeeping. We needed an early wakeup call, but couldn't use the clock radio alarm as there wasn't a radio in the room. There was no TV channel listing on the remote, or on the TV menu making viewing a chore.The TV remote had to be returned when checking-out. This place served its purpose, but not a place to revisit."

In [52]:
#ss=""".  Breakfast was the"""
#s = [spacy_tok(ss)]
#t = TEXT.numericalize(s)
#' '.join(s[0])
m3m = m3.model 
m4m = m4.model
m5m = m5.model
m3m[0].bs = 1
m4m[0].bs = 1
m5m[0].bs = 1
for i in range(tst.values[:,1].shape[0]):
    ss = tst["Description"][i]
    s = [spacy_tok(ss)]
    t = TEXT.numericalize(s)
   
    m3m.eval()
    m4m.eval()
    m5m.eval()
    m3m.reset()
    m4m.reset()
    m5m.reset()
    resm3m,*_ = m3m(t)
    resm4m,*_ = m4m(t)
    resm5m,*_ = m5m(t)
    
    res = resm3m+resm4m+resm5m
    user_id = tst["User_ID"][i]
    is_response = PH_LABEL.vocab.itos[to_np(torch.topk(res[-1], 1)[1])[0]]
    submission = submission.append({"User_ID":user_id, 'Is_Response':is_response.replace(" ","_")},ignore_index=True)

In [45]:
submission.head()

,User_ID,Is_Response
0,id80132,not_happy
1,id80133,happy
2,id80134,not_happy
3,id80135,not_happy
4,id80136,happy


In [53]:
submission.to_csv(PATH+"PredictHappinessModel_17_11_28_04_ensemble.csv", index=False)